In [21]:
import pickle
import importlib

from motor.motor_asyncio import AsyncIOMotorClient
from pymongo.server_api import ServerApi
import bokeh.models
import bokeh.plotting
import bokeh.io
import pandas as pd

import sp_project.data_scraping.openweather_prediction_scraper as weatherprediction
from sp_project.data_preparation.db_client import get_global_db_client
from sp_project.notebook_tools import default_clients
import sp_project.app_state
from sp_project.data_preparation.model import prepare_prediction_features, energy_prediction
from sp_project.data_preparation.prediction import fetch_prediction_daily

bokeh.io.output_notebook()

Loading BokehJS ...

In [22]:
lon = 7.70
lat = 46.42

In [23]:
with open("../../src/sp_project/backend_server/assets/prediction-model.pickle", "rb") as fh:
    model = pickle.load(fh)

In [24]:
async with default_clients() as (OWclient, db):
    app_state = sp_project.app_state.AppState(ow_client=OWclient, db_client=db, model=model)
    result = await fetch_prediction_daily(app_state, lon=lon, lat=lat)
    features = prepare_prediction_features(result, lat)
    prediction = energy_prediction(app_state.model, features)

In [25]:
prediction.head()

,wind,solar,nuclear,water_reservoir,water_river,water_pump,total
dt,,,,,,,
2023-05-06 11:00:00+00:00,-11.656665,776.354735,1006.361466,2470.604510,592.376608,1157.739232,5981.828377
2023-05-07 11:00:00+00:00,-22.195298,938.872697,554.723038,3191.629449,765.915672,1428.792896,6844.129307
2023-05-08 11:00:00+00:00,4.773768,480.733107,1356.144896,1840.364673,496.027111,947.939722,5116.628020
2023-05-09 11:00:00+00:00,9.283370,188.212870,1632.042678,1374.797223,362.469382,843.936522,4400.193086
2023-05-10 11:00:00+00:00,37.691917,-8.721736,2206.216163,384.902262,236.417645,415.465030,3267.404110


In [27]:
import bokeh.layouts # import column
import bokeh.models # import ColumnDataSource, RangeTool
import bokeh.plotting # import figure, show
import bokeh.palettes

source = bokeh.models.ColumnDataSource(data=prediction)


p = bokeh.plotting.figure(
    height=300, 
    width=800, 
    x_axis_type="datetime", 
    x_axis_location="above", 
    # tools="xpan", 
    # toolbar_location=None,
    x_range=(prediction.index[0], prediction.index[-1]),
)


p.yaxis.axis_label = 'Energy Production [MW]'

for col_name, color in zip(prediction.columns, bokeh.palettes.Colorblind[df_raw.shape[1]]):
    p.line('dt', col_name, source=source, line_color=color)

fig = p

bokeh.plotting.show(fig)

In [ ]:
import bokeh.embed
import json

with open("test.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(fig), fh)